In [764]:
#imports needed packages

import pandas as pd
import numpy as np

In [765]:
#sets directory

path='/Users/caseycloutier/Desktop/ECO688/Raw Data'

In [766]:
#import school modality dataset 

modality=pd.read_stata(path+'/schools_county.dta')

In [767]:
#convert to integer to remove decimals
modality['year']=modality['year'].astype(int)
modality['month']=modality['month'].astype(int)

#convert to string so they can be combined in 1 date column
modality['year']=modality['year'].astype(str)
modality['month']=modality['month'].astype(str)

In [768]:
#combines month and year in new variable 'date'

modality['date']=modality['month']+'/'+modality['year']

In [769]:
#recognize date columns as dates

modality['date']=pd.to_datetime(modality['date'], errors='coerce')

In [770]:
#drop rows with missing values from the dataset

modality=modality.dropna()

In [771]:
#create new variable countyname to identify county and state

modality['CountyName']=modality['county_name']+', '+modality['state_abb']

In [772]:
#drop obs with dates that aren't present in other datasets

start_date='2020-01-01'
end_date='2021-12-01'

mask=(modality['date'] >= start_date) & (modality['date'] <= end_date)

modality=modality.loc[mask]

In [773]:
#import service employment datasets

svc20=pd.read_csv(path+'/employment_industry/2020.q1-q4 102 102 Service-providing.csv')
svc21=pd.read_csv(path+'/employment_industry/2021.q1-q4 102 102 Service-providing.csv')

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [774]:
svc20=svc20[['area_fips','agglvl_code','area_title','own_code','year', 'qtr','month1_emplvl', 'month2_emplvl', 'month3_emplvl']]

svc20=svc20.melt(['area_fips','agglvl_code','area_title','own_code','year', 'qtr'], var_name='qtrmonth', value_name='emplvl')

In [775]:
svc20=svc20.loc[svc20['agglvl_code'] == 72]

In [776]:
svc20=svc20.groupby(['area_title','year','qtr','qtrmonth'])['emplvl'].sum().reset_index()

In [777]:
svc20['qtrmonth']=svc20['qtrmonth'].astype(str)
svc20['qtr']=svc20['qtr'].astype(str)

In [778]:
def month(svc20):    
    if (svc20['qtr']=='1') & (svc20['qtrmonth']=='month1_emplvl'):
        return '2020-01-01'
    elif (svc20['qtr']=='1') & (svc20['qtrmonth']=='month2_emplvl'):
        return '2020-02-01'
    elif (svc20['qtr']=='1') & (svc20['qtrmonth']=='month3_emplvl'):
        return '2020-03-01'
    elif (svc20['qtr']=='2') & (svc20['qtrmonth']=='month1_emplvl'):
        return '2020-04-01'
    elif (svc20['qtr']=='2') & (svc20['qtrmonth']=='month2_emplvl'):
        return '2020-05-01'
    elif (svc20['qtr']=='2') & (svc20['qtrmonth']=='month3_emplvl'):
        return '2020-06-01'
    elif (svc20['qtr']=='3') & (svc20['qtrmonth']=='month1_emplvl'):
        return '2020-07-01'
    elif (svc20['qtr']=='3') & (svc20['qtrmonth']=='month2_emplvl'):
        return '2020-08-01'
    elif (svc20['qtr']=='3') & (svc20['qtrmonth']=='month3_emplvl'):
        return '2020-09-01'
    elif (svc20['qtr']=='4') & (svc20['qtrmonth']=='month1_emplvl'):
        return '2020-10-01'
    elif (svc20['qtr']=='4') & (svc20['qtrmonth']=='month2_emplvl'):
        return '2020-11-01'
    elif (svc20['qtr']=='4') & (svc20['qtrmonth']=='month3_emplvl'):
        return '2020-12-01'

In [779]:
svc20['month']=svc20.apply(month, axis = 1)

In [780]:
names=svc20['area_title'].str.split(', ', expand=True)

names['State']=names[1].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})

names['CountyName']=names[0].str[:-7]
names['CountyName']=names['CountyName']+', '+names['State'] 

In [781]:
svc20['area_title']=names['CountyName']

In [782]:
svc20['area_title']=svc20['area_title'].replace('County', '')
svc20['area_title']=svc20['area_title'].astype(str)

In [783]:
svc20=svc20[~svc20['area_title'].str.endswith('Puerto Rico')]
svc20=svc20[~svc20['area_title'].str.endswith('Virgin Islands')]
svc20=svc20[~svc20['area_title'].str.endswith('Guam')]

In [784]:
svc20['area_title']=svc20['area_title'].str.upper()

In [785]:
svc20=svc20[['area_title','month','emplvl']]

In [786]:
svc21=svc21[['area_fips','agglvl_code','area_title','own_code','year', 'qtr','month1_emplvl', 'month2_emplvl', 'month3_emplvl']]

svc21=svc21.melt(['area_fips','agglvl_code','area_title','own_code','year', 'qtr'], var_name='qtrmonth', value_name='emplvl')

svc21=svc21.loc[svc21['agglvl_code'] == 72]

svc21=svc21.groupby(['area_title','year','qtr','qtrmonth'])['emplvl'].sum().reset_index()

In [787]:
svc21['qtrmonth']=svc21['qtrmonth'].astype(str)
svc21['qtr']=svc21['qtr'].astype(str)

In [788]:
def month21(svc21):    
    if (svc21['qtr']=='1') & (svc21['qtrmonth']=='month1_emplvl'):
        return '2021-01-01'
    elif (svc21['qtr']=='1') & (svc21['qtrmonth']=='month2_emplvl'):
        return '2021-02-01'
    elif (svc21['qtr']=='1') & (svc21['qtrmonth']=='month3_emplvl'):
        return '2021-03-01'
    elif (svc21['qtr']=='2') & (svc21['qtrmonth']=='month1_emplvl'):
        return '2021-04-01'
    elif (svc21['qtr']=='2') & (svc21['qtrmonth']=='month2_emplvl'):
        return '2021-05-01'
    elif (svc21['qtr']=='2') & (svc21['qtrmonth']=='month3_emplvl'):
        return '2021-06-01'
    elif (svc21['qtr']=='3') & (svc21['qtrmonth']=='month1_emplvl'):
        return '2021-07-01'
    elif (svc21['qtr']=='3') & (svc21['qtrmonth']=='month2_emplvl'):
        return '2021-08-01'
    elif (svc21['qtr']=='3') & (svc21['qtrmonth']=='month3_emplvl'):
        return '2021-09-01'
    elif (svc21['qtr']=='4') & (svc21['qtrmonth']=='month1_emplvl'):
        return '2021-10-01'
    elif (svc21['qtr']=='4') & (svc21['qtrmonth']=='month2_emplvl'):
        return '2021-11-01'
    elif (svc21['qtr']=='4') & (svc21['qtrmonth']=='month3_emplvl'):
        return '2021-12-01'

In [789]:
svc21['month']=svc21.apply(month21, axis = 1)

In [790]:
names=svc21['area_title'].str.split(', ', expand=True)

names21['State']=names21[1].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})

names21['CountyName']=names21[0].str[:-7]
names21['CountyName']=names21['CountyName']+', '+names21['State'] 

svc21['area_title']=names21['CountyName']

svc21['area_title']=svc21['area_title'].replace('County', '')
svc21['area_title']=svc21['area_title'].astype(str)

svc21=svc21[~svc21['area_title'].str.endswith('Puerto Rico')]
svc21=svc21[~svc21['area_title'].str.endswith('Virgin Islands')]
svc21=svc21[~svc21['area_title'].str.endswith('Guam')]

svc21['area_title']=svc21['area_title'].str.upper()

In [791]:
svc21=svc21[~svc21['area_title'].str.endswith('Puerto Rico')]
svc21=svc21[~svc21['area_title'].str.endswith('Virgin Islands')]
svc21=svc21[~svc21['area_title'].str.endswith('Guam')]

svc21['area_title']=svc21['area_title'].str.upper()

svc21=svc21[['area_title','month','emplvl']]

In [792]:
svc20['area_title']=svc20['area_title'].replace(['ANCHORAGE MUNIC, AK','ALEUTIANS EAST , AK','ALEUTIANS WEST CENS, AK','BALTIMO, MD','BETHEL CENS, AK','BRISTOL BAY , AK','BRIST, VA','BUENA VIS, VA','CHARLOTTESVIL, VA','CHESAPEA, VA','COLONIAL HEIGH, VA','COVINGT, VA','DENALI , AK','DILLINGHAM CENS, AK','FAIRBANKS NORTH STAR , AK','HAINES , AK','JUNEAU CITY AND , AK','KENAI PENINSULA , AK','KETCHIKAN GATEWAY , AK','KODIAK ISLAND , AK','ALEXANDR, VA','COVINGT, VA','DANVIL, VA','FAIRF, VA','FRANKL, VA','FREDERICKSBU, VA','GAL, VA','HAMPT, VA','HARRISONBU, VA','HOPEWE, VA','LEXINGT, VA','LYNCHBU, VA','MANASS, VA','MANASSAS PA, VA','MARTINSVIL, VA','NEWPORT NE, VA','NORFO, VA','NORT, VA','PETERSBU, VA','POQUOS, VA','PORTSMOU, VA','RADFO, VA','RICHMO, VA','ROANO, VA','SAL, VA','STAUNT, VA','SUFFO, VA','VIRGINIA BEA, VA','WAYNESBO, VA','WINCHEST, VA'],['ALEUTIANS EAST, AK','ALEUTIANS WEST, AK','ANCHORAGE, AK','BALTIMORE CITY, MD','BETHEL, AK','BRISTOL BAY, AK','BRISTOL CITY, VA','BUENA VISTA CITY, VA ','CHARLOTTESVILLE CITY, VA','CHESAPEAKE CITY, VA','COLONIAL HEIGHTS CITY, VA','COVINGTON CITY, VA','DENALI, AK','DILLINGHAM, AK','FAIRBANKS NORTH STAR, AK','HAINES, AK','JUNEAU, AK','KENAI PENINSULA, AK','KETCHIKAN GATEWAY, AK','KODIAK ISLAND, AK','ALEXANDRIA CITY, VA','COVINGTON CITY, VA','DANVILLE CITY, VA','FAIRFAX CITY, VA','FRANKLIN CITY, VA','FREDERICKSBURG CITY, VA','GALAX CITY, VA','HAMPTON CITY, VA','HARRISONBURG CITY, VA','HOPEWELL CITY, VA','LEXINGTON CITY, VA','LYNCHBURG CITY, VA','MANASSAS CITY, VA','MANASSAS PARK CITY, VA','MARTINSVILLE, VA','NEWPORT NEWS CITY, VA','NORFOLK CITY, VA','NORTON CITY, VA','PETERSBURG CITY, VA','POQUOSON CITY, VA','PORTSMOUTH CITY, VA','RADFORD CITY, VA','RICHMOND CITY, VA','ROANOKE CITY, VA','SALEM CITY, VA','STAUNTON CITY, VA','SUFFOLK CITY, VA','VIRGINIA BEACH CITY, VA','WAYNESBORO CITY, VA','WINCHESTER CITY, VA'])

In [793]:
svc21['area_title']=svc21['area_title'].replace(['ANCHORAGE MUNIC, AK','ALEUTIANS EAST , AK','ALEUTIANS WEST CENS, AK','BALTIMO, MD','BETHEL CENS, AK','BRISTOL BAY , AK','BRIST, VA','BUENA VIS, VA','CHARLOTTESVIL, VA','CHESAPEA, VA','COLONIAL HEIGH, VA','COVINGT, VA','DENALI , AK','DILLINGHAM CENS, AK','FAIRBANKS NORTH STAR , AK','HAINES , AK','JUNEAU CITY AND , AK','KENAI PENINSULA , AK','KETCHIKAN GATEWAY , AK','KODIAK ISLAND , AK','ALEXANDR, VA','COVINGT, VA','DANVIL, VA','FAIRF, VA','FRANKL, VA','FREDERICKSBU, VA','GAL, VA','HAMPT, VA','HARRISONBU, VA','HOPEWE, VA','LEXINGT, VA','LYNCHBU, VA','MANASS, VA','MANASSAS PA, VA','MARTINSVIL, VA','NEWPORT NE, VA','NORFO, VA','NORT, VA','PETERSBU, VA','POQUOS, VA','PORTSMOU, VA','RADFO, VA','RICHMO, VA','ROANO, VA','SAL, VA','STAUNT, VA','SUFFO, VA','VIRGINIA BEA, VA','WAYNESBO, VA','WINCHEST, VA'],['ALEUTIANS EAST, AK','ALEUTIANS WEST, AK','ANCHORAGE, AK','BALTIMORE CITY, MD','BETHEL, AK','BRISTOL BAY, AK','BRISTOL CITY, VA','BUENA VISTA CITY, VA ','CHARLOTTESVILLE CITY, VA','CHESAPEAKE CITY, VA','COLONIAL HEIGHTS CITY, VA','COVINGTON CITY, VA','DENALI, AK','DILLINGHAM, AK','FAIRBANKS NORTH STAR, AK','HAINES, AK','JUNEAU, AK','KENAI PENINSULA, AK','KETCHIKAN GATEWAY, AK','KODIAK ISLAND, AK','ALEXANDRIA CITY, VA','COVINGTON CITY, VA','DANVILLE CITY, VA','FAIRFAX CITY, VA','FRANKLIN CITY, VA','FREDERICKSBURG CITY, VA','GALAX CITY, VA','HAMPTON CITY, VA','HARRISONBURG CITY, VA','HOPEWELL CITY, VA','LEXINGTON CITY, VA','LYNCHBURG CITY, VA','MANASSAS CITY, VA','MANASSAS PARK CITY, VA','MARTINSVILLE, VA','NEWPORT NEWS CITY, VA','NORFOLK CITY, VA','NORTON CITY, VA','PETERSBURG CITY, VA','POQUOSON CITY, VA','PORTSMOUTH CITY, VA','RADFORD CITY, VA','RICHMOND CITY, VA','ROANOKE CITY, VA','SALEM CITY, VA','STAUNTON CITY, VA','SUFFOLK CITY, VA','VIRGINIA BEACH CITY, VA','WAYNESBORO CITY, VA','WINCHESTER CITY, VA'])

In [794]:
svc20=svc20[~svc20['area_title'].str.startswith('CHUGACH CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('COPPER RIVER CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('CRAIG, VA')]
svc20=svc20[~svc20['area_title'].str.startswith('HOONAH-ANGOON CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('KUSILVAK CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('LAKE AND PENINSULA , AK')]
svc20=svc20[~svc20['area_title'].str.startswith('MATANUSKA-SUSITNA , AK')]
svc20=svc20[~svc20['area_title'].str.startswith('NOME CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('NORTH SLOPE , AK')]
svc20=svc20[~svc20['area_title'].str.startswith('PETERSBURG , AK')]
svc20=svc20[~svc20['area_title'].str.startswith('PRINCE OF WALES-HYDER CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('SITKA CITY AND , AK')]
svc20=svc20[~svc20['area_title'].str.startswith('SKAGWAY MUNIC, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('SOUTHEAST FAIRBANKS CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('YAKUTAT CITY AND , AK')]
svc20=svc20[~svc20['area_title'].str.startswith('YUKON-KOYUKUK CENS, AK')]
svc20=svc20[~svc20['area_title'].str.startswith('UNKNOWN')]

In [795]:
svc21=svc21[~svc21['area_title'].str.startswith('CHUGACH CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('COPPER RIVER CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('CRAIG, VA')]
svc21=svc21[~svc21['area_title'].str.startswith('HOONAH-ANGOON CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('KUSILVAK CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('LAKE AND PENINSULA , AK')]
svc21=svc21[~svc21['area_title'].str.startswith('MATANUSKA-SUSITNA , AK')]
svc21=svc21[~svc21['area_title'].str.startswith('NOME CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('NORTH SLOPE , AK')]
svc21=svc21[~svc21['area_title'].str.startswith('PETERSBURG , AK')]
svc21=svc21[~svc21['area_title'].str.startswith('PRINCE OF WALES-HYDER CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('SITKA CITY AND , AK')]
svc21=svc21[~svc21['area_title'].str.startswith('SKAGWAY MUNIC, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('SOUTHEAST FAIRBANKS CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('YAKUTAT CITY AND , AK')]
svc21=svc21[~svc21['area_title'].str.startswith('YUKON-KOYUKUK CENS, AK')]
svc21=svc21[~svc21['area_title'].str.startswith('UNKNOWN')]

In [796]:
frames=[svc20,svc21]
svcemp=pd.concat(frames)

In [797]:
svcemp=svcemp.rename(columns={'month':'date'})
svcemp=svcemp.sort_values(['area_title', 'date'])

In [798]:
modality=modality.rename(columns={'CountyName':'area_title'})

In [799]:
modality=modality.sort_values(['area_title','date'])

In [800]:
modality=modality[['area_title','date','number_schools', 'share_all_closed_50', 'share_all_closed_25',
       'share_all_closed_75', 'share_elem_closed_50', 'share_elem_closed_25',
       'share_elem_closed_75', 'share_middlehigh_closed_50',
       'share_middlehigh_closed_25', 'share_middlehigh_closed_75',
       'mean_all_change', 'mean_elem_change', 'mean_middlehigh_change',
       'total_students']]

In [801]:
svcemp['date']=svcemp['date'].astype(str)
modality['date']=modality['date'].astype(str)

In [802]:
data=pd.merge(svcemp, modality,  how='left', left_on=['date','area_title'], right_on = ['date','area_title'])

In [803]:
controls=pd.read_csv('/Users/caseycloutier/Desktop/ECO688/Raw Data/eco400.csv')

In [804]:
controls=controls[['CountyName_x','Date','Year','ALWAYS','Completeness_pct','Q_GDP','trump_True','Percent_Female',
       'Percent_Black', 'Percent_AmerInd', 'Percent_Asian', 'Percent_PacI','Percent_HisLat']]

In [805]:
namesc=controls['CountyName_x'].str.split(', ', expand=True)

namesc['State']=namesc[1]

namesc['CountyName']=namesc[0].str[:-7]
namesc['CountyName']=namesc['CountyName']+', '+namesc['State'] 

namesc['area_title']=namesc['CountyName']

namesc['area_title']=namesc['area_title'].replace('County', '')
namesc['area_title']=namesc['area_title'].astype(str)

In [806]:
namesc['area_title']=namesc['area_title'].str.upper()

In [807]:
controls['area_title']=namesc['area_title']

In [808]:
controls['date']=controls['Date'].replace(['2020-01-31','2020-02-29','2020-03-31','2020-04-30','2020-05-31','2020-06-30','2020-07-31','2020-08-31','2020-09-30','2020-10-31','2020-11-30','2020-12-31','2021-01-31','2021-02-28','2021-03-31','2021-04-30','2021-05-31','2021-06-30','2021-07-31','2021-08-31','2021-09-30','2021-10-31','2021-11-30','2021-12-31'],['2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01','2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01','2021-01-01','2021-02-01','2021-03-01','2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01','2021-09-01','2021-10-01','2021-11-01','2021-12-01'])

In [809]:
data=pd.merge(data, controls,  how='left', left_on=['date','area_title'], right_on = ['date','area_title'])

In [810]:
population=pd.read_excel(path+'/population.xlsx')

In [811]:
population['County']=population['County'].str[1:]

In [812]:
population=population.melt(['County'], var_name='Year', value_name='Population')
population=population.sort_values(['County'])

In [813]:
namespop=population['County'].str.split(', ', expand=True)

namespop['State']=namespop[1]

namespop['area_title']=namespop[0].str[:-7]
namespop['State']=namespop[1].replace({'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY','Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO','Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD', 'Tennessee':'TN','Texas':'TX', 'Utah':'UT','Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV','Wisconsin':'WI', 'Wyoming':'WY','District of Columbia':'DC'})

In [814]:
namespop['area_title']=namespop['area_title']+', '+namespop['State'] 

namespop['area_title']=namespop['area_title'].astype(str)

In [815]:
population['area_title']=namespop['area_title'].str.upper()
population

,County,Year,Population,area_title
2317,"Abbeville County, South Carolina",2020,24212,"ABBEVILLE, SC"
5460,"Abbeville County, South Carolina",2021,24299,"ABBEVILLE, SC"
1114,"Acadia Parish, Louisiana",2020,57455,"ACADIA, LA"
4257,"Acadia Parish, Louisiana",2021,57288,"ACADIA, LA"
2821,"Accomack County, Virginia",2020,33348,"ACCOMACK, VA"
...,...,...,...,...
5919,"Zapata County, Texas",2021,13908,"ZAPATA, TX"
2777,"Zavala County, Texas",2020,9652,"ZAVALA, TX"
5920,"Zavala County, Texas",2021,9534,"ZAVALA, TX"
2428,"Ziebach County, South Dakota",2020,2388,"ZIEBACH, SD"


In [816]:
population=population[['area_title','Year','Population']]

In [817]:
population['area_title']=population['area_title'].replace(['ANCHORAGE MUNIC, AK','ALEUTIANS EAST , AK','ALEUTIANS WEST CENS, AK','BALTIMO, MD','BETHEL CENS, AK','BRISTOL BAY , AK','BRIST, VA','BUENA VIS, VA','CHARLOTTESVIL, VA','CHESAPEA, VA','COLONIAL HEIGH, VA','COVINGT, VA','DENALI , AK','DILLINGHAM CENS, AK','FAIRBANKS NORTH STAR , AK','HAINES , AK','JUNEAU CITY AND , AK','KENAI PENINSULA , AK','KETCHIKAN GATEWAY , AK','KODIAK ISLAND , AK','ALEXANDR, VA','COVINGT, VA','DANVIL, VA','FAIRF, VA','FRANKL, VA','FREDERICKSBU, VA','GAL, VA','HAMPT, VA','HARRISONBU, VA','HOPEWE, VA','LEXINGT, VA','LYNCHBU, VA','MANASS, VA','MANASSAS PA, VA','MARTINSVIL, VA','NEWPORT NE, VA','NORFO, VA','NORT, VA','PETERSBU, VA','POQUOS, VA','PORTSMOU, VA','RADFO, VA','RICHMO, VA','ROANO, VA','SAL, VA','STAUNT, VA','SUFFO, VA','VIRGINIA BEA, VA','WAYNESBO, VA','WINCHEST, VA'],['ALEUTIANS EAST, AK','ALEUTIANS WEST, AK','ANCHORAGE, AK','BALTIMORE CITY, MD','BETHEL, AK','BRISTOL BAY, AK','BRISTOL CITY, VA','BUENA VISTA CITY, VA ','CHARLOTTESVILLE CITY, VA','CHESAPEAKE CITY, VA','COLONIAL HEIGHTS CITY, VA','COVINGTON CITY, VA','DENALI, AK','DILLINGHAM, AK','FAIRBANKS NORTH STAR, AK','HAINES, AK','JUNEAU, AK','KENAI PENINSULA, AK','KETCHIKAN GATEWAY, AK','KODIAK ISLAND, AK','ALEXANDRIA CITY, VA','COVINGTON CITY, VA','DANVILLE CITY, VA','FAIRFAX CITY, VA','FRANKLIN CITY, VA','FREDERICKSBURG CITY, VA','GALAX CITY, VA','HAMPTON CITY, VA','HARRISONBURG CITY, VA','HOPEWELL CITY, VA','LEXINGTON CITY, VA','LYNCHBURG CITY, VA','MANASSAS CITY, VA','MANASSAS PARK CITY, VA','MARTINSVILLE, VA','NEWPORT NEWS CITY, VA','NORFOLK CITY, VA','NORTON CITY, VA','PETERSBURG CITY, VA','POQUOSON CITY, VA','PORTSMOUTH CITY, VA','RADFORD CITY, VA','RICHMOND CITY, VA','ROANOKE CITY, VA','SALEM CITY, VA','STAUNTON CITY, VA','SUFFOLK CITY, VA','VIRGINIA BEACH CITY, VA','WAYNESBORO CITY, VA','WINCHESTER CITY, VA'])

<ipython-input-817-d250b68eb371>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['area_title']=population['area_title'].replace(['ANCHORAGE MUNIC, AK','ALEUTIANS EAST , AK','ALEUTIANS WEST CENS, AK','BALTIMO, MD','BETHEL CENS, AK','BRISTOL BAY , AK','BRIST, VA','BUENA VIS, VA','CHARLOTTESVIL, VA','CHESAPEA, VA','COLONIAL HEIGH, VA','COVINGT, VA','DENALI , AK','DILLINGHAM CENS, AK','FAIRBANKS NORTH STAR , AK','HAINES , AK','JUNEAU CITY AND , AK','KENAI PENINSULA , AK','KETCHIKAN GATEWAY , AK','KODIAK ISLAND , AK','ALEXANDR, VA','COVINGT, VA','DANVIL, VA','FAIRF, VA','FRANKL, VA','FREDERICKSBU, VA','GAL, VA','HAMPT, VA','HARRISONBU, VA','HOPEWE, VA','LEXINGT, VA','LYNCHBU, VA','MANASS, VA','MANASSAS PA, VA','MARTINSVIL, VA','NEWPORT NE,

In [818]:
population=population[~population['area_title'].str.startswith('CHUGACH CENS, AK')]
population=population[~population['area_title'].str.startswith('COPPER RIVER CENS, AK')]
population=population[~population['area_title'].str.startswith('CRAIG, VA')]
population=population[~population['area_title'].str.startswith('HOONAH-ANGOON CENS, AK')]
population=population[~population['area_title'].str.startswith('LAKE AND PENINSULA , AK')]
population=population[~population['area_title'].str.startswith('MATANUSKA-SUSITNA , AK')]
population=population[~population['area_title'].str.startswith('NOME CENS, AK')]
population=population[~population['area_title'].str.startswith('NORTH SLOPE , AK')]
population=population[~population['area_title'].str.startswith('PETERSBURG , AK')]
population=population[~population['area_title'].str.startswith('PRINCE OF WALES-HYDER CENS, AK')]
population=population[~population['area_title'].str.startswith('SOUTHEAST FAIRBANKS CENS, AK')]
population=population[~population['area_title'].str.startswith('YAKUTAT CITY AND , AK')]
population=population[~population['area_title'].str.startswith('YUKON-KOYUKUK CENS, AK')]

In [819]:
data=pd.merge(data, population,  how='left', left_on=['area_title','Year'], right_on = ['area_title','Year'])

In [820]:
data=data[data['Population'].notna()]

In [821]:
data=data[~data['area_title'].str.endswith('NE')]
data=data[~data['area_title'].str.endswith('SD')]
data=data[~data['area_title'].str.endswith('UT')]
data=data[~data['area_title'].str.endswith('MT')]

data=data[~data['area_title'].str.endswith('ALCONA, MI')]
data=data[~data['area_title'].str.endswith('AMITE, MS')]
data=data[~data['area_title'].str.endswith('BAILEY, TX')]
data=data[~data['area_title'].str.endswith('BAMBERG, SC')]
data=data[~data['area_title'].str.endswith('BUTTE, ID')]
data=data[~data['area_title'].str.endswith('CALDWELL, LA')]
data=data[~data['area_title'].str.endswith('CARS, NV')]
data=data[~data['area_title'].str.endswith('CARTER, MO')]
data=data[~data['area_title'].str.endswith('CASEY, KY')]
data=data[~data['area_title'].str.endswith('CHATTAHOOCHEE, GA')]
data=data[~data['area_title'].str.endswith('CLEAR CREEK, CO')]
data=data[~data['area_title'].str.endswith('CONCHO, TX')]
data=data[~data['area_title'].str.endswith('CRAWFORD, IN')]
data=data[~data['area_title'].str.endswith('CROCKETT, TX')]
data=data[~data['area_title'].str.endswith('CUMBERLAND, KY')]
data=data[~data['area_title'].str.endswith('CUSTER, CO')]
data=data[~data['area_title'].str.endswith('DE BACA, NM')]
data=data[~data['area_title'].str.endswith('DECATUR, TN')]
data=data[~data['area_title'].str.endswith('ELLIOTT, KY')]
data=data[~data['area_title'].str.endswith('EMPOR, VA')]
data=data[~data['area_title'].str.endswith('EUREKA, NV')]
data=data[~data['area_title'].str.endswith('FALLS CHUR, VA')]
data=data[~data['area_title'].str.endswith('FERRY, WA')]
data=data[~data['area_title'].str.endswith('FOSTER, ND')]
data=data[~data['area_title'].str.endswith('GALLATIN, IL')]
data=data[~data['area_title'].str.endswith('GILPIN, CO')]
data=data[~data['area_title'].str.endswith('GLASCOCK, GA')]
data=data[~data['area_title'].str.endswith('GREENLEE, AZ')]
data=data[~data['area_title'].str.endswith('HAMILTON, FL')]
data=data[~data['area_title'].str.endswith('HAMILTON, IL')]
data=data[~data['area_title'].str.endswith('HANCOCK, GA')]
data=data[~data['area_title'].str.endswith('HANCOCK, TN')]
data=data[~data['area_title'].str.endswith('HARDIN, IL')]
data=data[~data['area_title'].str.endswith('HARDING, NM')]
data=data[~data['area_title'].str.endswith('HENDERSON, IL')]
data=data[~data['area_title'].str.endswith('ISSAQUENA, MS')]
data=data[~data['area_title'].str.endswith('JEFFERSON, MS')]
data=data[~data['area_title'].str.endswith('JIM HOGG, TX')]
data=data[~data['area_title'].str.endswith('JOHNSON, GA')]
data=data[~data['area_title'].str.endswith('JONES, NC')]
data=data[~data['area_title'].str.endswith('KENEDY, TX')]
data=data[~data['area_title'].str.endswith('KEWEENAW, MI')]
data=data[~data['area_title'].str.endswith('KIMBLE, TX')]
data=data[~data['area_title'].str.endswith('LAKE OF THE WOODS, MN')]
data=data[~data['area_title'].str.endswith('LASALLE, LA')]
data=data[~data['area_title'].str.endswith('LEE, KY')]
data=data[~data['area_title'].str.endswith('LEWIS, WV')]
data=data[~data['area_title'].str.endswith('LINCOLN, KS')]
data=data[~data['area_title'].str.endswith('LOVING, TX')]
data=data[~data['area_title'].str.endswith('MACON, GA')]
data=data[~data['area_title'].str.endswith('MASON, TX')]
data=data[~data['area_title'].str.endswith('MCCORMICK, SC')]
data=data[~data['area_title'].str.endswith('MEIGS, TN')]
data=data[~data['area_title'].str.endswith('MENOMINEE, WI')]
data=data[~data['area_title'].str.endswith('MILLER, GA')]
data=data[~data['area_title'].str.endswith('MINERAL, CO')]
data=data[~data['area_title'].str.endswith('MISSAUKEE, MI')]
data=data[~data['area_title'].str.endswith('MONTGOMERY, GA')]
data=data[~data['area_title'].str.endswith('MOORE, TN')]
data=data[~data['area_title'].str.endswith('NANTUCKET, MA')]
data=data[~data['area_title'].str.endswith('NOTTOWAY, VA')]
data=data[~data['area_title'].str.endswith('NOXUBEE, MS')]
data=data[~data['area_title'].str.endswith('POCAHONTAS, WV')]
data=data[~data['area_title'].str.endswith('POPE, IL')]
data=data[~data['area_title'].str.endswith('PRINCE EDWARD, VA')]
data=data[~data['area_title'].str.endswith('QUITMAN, GA')]
data=data[~data['area_title'].str.endswith('QUITMAN, MS')]
data=data[~data['area_title'].str.endswith('RALLS, MO')]
data=data[~data['area_title'].str.endswith('RAPPAHANNOCK, VA')]
data=data[~data['area_title'].str.endswith('RICHLAND, IL')]
data=data[~data['area_title'].str.endswith('RICHMOND, VA')]
data=data[~data['area_title'].str.endswith('ROBERTSON, KY')]
data=data[~data['area_title'].str.endswith('ROCKBRIDGE, VA')]
data=data[~data['area_title'].str.endswith('SAN JUAN, CO')]
data=data[~data['area_title'].str.endswith('SCHLEICHER, TX')]
data=data[~data['area_title'].str.endswith('SCHOOLCRAFT, MI')]
data=data[~data['area_title'].str.endswith('SHARP, AR')]
data=data[~data['area_title'].str.endswith('SHERMAN, KS')]
data=data[~data['area_title'].str.endswith('SHERMAN, OR')]
data=data[~data['area_title'].str.endswith('SIERRA, NM')]
data=data[~data['area_title'].str.endswith('SLOPE, ND')]
data=data[~data['area_title'].str.endswith('ST. LOU, MO')]
data=data[~data['area_title'].str.endswith('STANTON, KS')]
data=data[~data['area_title'].str.endswith('STE. GENEVIEVE, MO')]
data=data[~data['area_title'].str.endswith('STEWART, GA')]
data=data[~data['area_title'].str.endswith('SUSSEX, VA')]
data=data[~data['area_title'].str.endswith('TALLAHATCHIE, MS')]
data=data[~data['area_title'].str.endswith('TELFAIR, GA')]
data=data[~data['area_title'].str.endswith('TREGO, KS')]
data=data[~data['area_title'].str.endswith('TROUSDALE, TN')]
data=data[~data['area_title'].str.endswith('TWIGGS, GA')]
data=data[~data['area_title'].str.endswith('TYRRELL, NC')]
data=data[~data['area_title'].str.endswith('UPTON, TX')]
data=data[~data['area_title'].str.endswith('VAN BUREN, TN')]
data=data[~data['area_title'].str.endswith('WARREN, GA')]
data=data[~data['area_title'].str.endswith('WASHINGTON, NC')]
data=data[~data['area_title'].str.endswith('WAYNE, TN')]
data=data[~data['area_title'].str.endswith('WILKINSON, GA')]
data=data[~data['area_title'].str.endswith('WILKINSON, MS')]
data=data[~data['area_title'].str.endswith('WILLIAMSBU, VA')]
data=data[~data['area_title'].str.endswith('WOODSON, KS')]
data=data[~data['area_title'].str.endswith('DOLORES, CO')]

In [822]:
data.to_csv('/Users/caseycloutier/Desktop/ECO688/Cleaned Data/data.csv')